PANDAS

## 1. Lectura y Exploración Inicial

In [ ]:
import pandas as pd
import numpy as np

# Lectura CSV
df = pd.read_csv('archivo.csv')

# Info rapida
df.shape                    # (filas, columnas)
df.info()                   # tipos y NAs por columna
df.describe()               # estadisticas numericas
df.head(10)                 # primeras n filas
df.tail()                   # ultimas filas
df.columns                  # nombres de columnas
df.dtypes                   # tipos de datos

## 2. Limpieza de NAs - Por Columna

In [ ]:
# Detectar NAs por columna
df.isna().sum()                          # cuenta NAs por columna
df.isnull().sum()                        # alias de isna()

# Porcentaje de NAs por columna (TIPICO EN EXAMENES)
nrow = len(df)
pct_nas = (df.isna().sum() / nrow) * 100
print(pct_nas)

# Eliminar columnas con >= 25% de NAs (PATRON EXAMEN 2024)
threshold = 25
columnas_validas = pct_nas[pct_nas < threshold].index
df_clean = df[columnas_validas]

# Alternativa: eliminar columnas con cualquier NA
df_no_na_cols = df.dropna(axis=1)        # axis=1 -> columnas

# Eliminar columna especifica
df.drop('columna_name', axis=1, inplace=True)
df.drop(['col1', 'col2'], axis=1, inplace=True)

## 3. Limpieza de NAs - Por Fila

In [ ]:
# Eliminar filas con cualquier NA (PATRON EXAMENES)
df_clean = df.dropna(axis=0)             # axis=0 -> filas
df.dropna(inplace=True)                  # modifica el df original

# Eliminar filas donde TODAS sean NA
df_clean = df.dropna(how='all')

# Eliminar filas con NA en columnas especificas
df_clean = df.dropna(subset=['col1', 'col2'])

# IMPORTANTE: reset index despues de dropna
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)  # drop=True: no guardar index viejo

# Contar NAs por fila
df.isna().sum(axis=1)                    # axis=1 -> suma por fila

## 2b. TRAMPA: NAs como Strings

In [ ]:
# CUIDADO: valores como "N/A", "NA", "nan", "None" como STRINGS
# No los detecta df.isna() porque son strings validos!

# 1) Al leer CSV, especificar que valores son NA
df = pd.read_csv('archivo.csv', na_values=['N/A', 'NA', 'nan', 'None', '', ' '])

# 2) Si ya leiste el CSV, reemplazar strings por NaN real
import numpy as np
df.replace(['N/A', 'NA', 'nan', 'None', '', ' '], np.nan, inplace=True)

# 3) Despues de reemplazar, AHORA si usar dropna()
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# 4) Detectar estos valores string (antes de convertir)
# Ver si hay valores sospechosos
df['columna'].value_counts()  # revisar si aparece 'N/A', 'NA', etc.

# 5) Eliminar filas con valores especificos
valores_invalidos = ['N/A', 'NA', 'nan', 'None', '', 'Unkown', 'Unknown']
df_clean = df[~df['columna'].isin(valores_invalidos)]

# 6) Para TODAS las columnas tipo object (string)
for col in df.select_dtypes(include=['object']).columns:
    df[col].replace(['N/A', 'NA', 'nan', 'None', ''], np.nan, inplace=True)

## 2c. Workflow Completo de Limpieza

In [ ]:
# WORKFLOW TIPICO DE EXAMEN (orden recomendado)

# 1. Leer CSV con na_values
df = pd.read_csv('archivo.csv', na_values=['N/A', 'NA', 'nan', 'None', ''])

# 2. Seleccionar columnas de interes
cols = ['product', 'issue', 'company', 'state', 'timely_response']
df = df[cols]

# 3. Convertir strings NA residuales a NaN real
df.replace(['N/A', 'NA', 'Unknown', 'Unkown'], np.nan, inplace=True)

# 4. Eliminar columnas con muchos NAs (>= 25%)
pct_nas = (df.isna().sum() / len(df)) * 100
cols_validas = pct_nas[pct_nas < 25].index
df = df[cols_validas]

# 5. Eliminar filas con NAs
df.dropna(inplace=True)

# 6. Reset index
df.reset_index(drop=True, inplace=True)

# 7. Convertir booleanos a 0/1
df.replace({True: 1, False: 0}, inplace=True)

# 8. Verificar resultado
print(f'Shape final: {df.shape}')
print(f'NAs totales: {df.isna().sum().sum()}')
df.info()

## 4. Seleccion de Columnas por Tipo

In [ ]:
# Seleccionar solo columnas numericas (PATRON EXAMEN 2024)
numeric_cols = df.select_dtypes(include=[np.number]).columns
df_numeric = df[numeric_cols]

# Alternativa con tipos especificos
df_numeric = df.select_dtypes(include=['int64', 'float64'])

# Seleccionar solo columnas de texto
df_text = df.select_dtypes(include=['object'])

# Excluir tipos especificos
df_no_numeric = df.select_dtypes(exclude=[np.number])

# Seleccion manual de columnas (PATRON EXAMEN 2023)
cols_deseadas = ['product', 'issue', 'company', 'state']
df_subset = df[cols_deseadas]

# Lista de columnas
list(df.columns)

## 5. Reemplazo de Valores y Booleanos

In [ ]:
# Convertir booleanos a 0/1 (PATRON TODOS LOS EXAMENES)
df.replace({True: 1, False: 0}, inplace=True)

# Reemplazar valores especificos
df['columna'].replace('valor_viejo', 'valor_nuevo', inplace=True)

# Reemplazar multiples valores
df.replace({'valor1': 'nuevo1', 'valor2': 'nuevo2'}, inplace=True)

# Reemplazar por diccionario de columnas
df.replace({
    'col1': {'A': 1, 'B': 2},
    'col2': {'X': 10, 'Y': 20}
}, inplace=True)

# Rellenar NAs
df.fillna(0, inplace=True)               # con valor constante
df['col'].fillna(df['col'].mean(), inplace=True)  # con media
df.fillna(method='ffill')                # forward fill
df.fillna(method='bfill')                # backward fill

## 6. Filtrado de Filas - Condiciones

In [ ]:
# Filtro simple
df_filtrado = df[df['columna'] > 100]

# Eliminar filas con valor especifico (PATRON EXAMEN 2022)
df_clean = df[df['Total_Gross'] != 'Gross Unkown']

# Multiples condiciones con & (AND) y | (OR)
df_filtrado = df[(df['Year'] >= 1980) & (df['Year'] <= 2019)]
df_filtrado = df[(df['col'] == 'A') | (df['col'] == 'B')]

# Usar isin para filtrar por lista (PATRON TODOS LOS EXAMENES)
valores_validos = ['Action', 'Drama', 'Comedy']
df_filtrado = df[df['main_genre'].isin(valores_validos)]

# Negacion con ~
df_excluidos = df[~df['genre'].isin(['Horror', 'Thriller'])]

# Reset index despues de filtrar
df_filtrado.reset_index(drop=True, inplace=True)

## 7. Regex y Limpieza de Strings (re)

In [ ]:
import re

# Eliminar caracteres especificos con re.sub (PATRON EXAMEN 2022)
# Formato: re.sub(patron, reemplazo, string)
df['clean_col'] = df['col'].apply(lambda x: re.sub(r'\$', '', x))
df['clean_col'] = df['clean_col'].apply(lambda x: re.sub('M', '', x))

# Ejemplo completo: '$125.5M' -> 125.5
df['New_total_gross'] = (df['Total_Gross']
                         .apply(lambda x: re.sub(r'\$', '', x))
                         .apply(lambda x: re.sub('M', '', x))
                         .apply(lambda x: float(x)))

# Metodos de string basicos
df['col'].str.lower()                    # minusculas
df['col'].str.upper()                    # mayusculas
df['col'].str.strip()                    # eliminar espacios extremos
df['col'].str.replace('viejo', 'nuevo')  # reemplazar sin regex

# Buscar si contiene patron
df[df['col'].str.contains('patron', na=False)]

# Split de strings (EXAMEN 2022 - actores)
re.split(', ', 'Actor1, Actor2, Actor3')  # devuelve lista

## 8. Valores Unicos y Duplicados

In [ ]:
# Valores unicos
df['columna'].unique()                   # array de valores unicos
df['columna'].nunique()                  # numero de valores unicos
df['columna'].value_counts()             # conteo de cada valor

# Conteo con sort
df['columna'].value_counts().sort_values(ascending=False)

# Duplicados
df.duplicated()                          # bool por fila
df.duplicated().sum()                    # cuenta duplicados
df.drop_duplicates(inplace=True)         # elimina duplicados

# Duplicados basados en columnas especificas
df.drop_duplicates(subset=['col1', 'col2'], inplace=True)

# Ver valores unicos de multiples columnas
for col in df.columns:
    print(f'{col}: {df[col].nunique()} valores unicos')

## 9. Crear y Transformar Columnas

In [ ]:
# Crear columna nueva directamente
df['nueva_col'] = 1                      # valor constante
df['suma'] = df['col1'] + df['col2']     # operacion aritmetica

# Crear columna desde suma de otras (PATRON EXAMEN 01MIAR)
df['Relatives'] = df['SibSp'] + df['Parch']

# Con apply y lambda (PATRON EXAMEN 2022 - decadas)
def get_decade(year):
    return (year // 10) * 10

df['decade'] = df['Year'].apply(get_decade)
# Alternativa con lambda
df['decade'] = df['Year'].apply(lambda x: (x // 10) * 10)

# Apply con multiples columnas
df['ratio'] = df.apply(lambda row: row['col1'] / row['col2'], axis=1)

# Apply con if/else
df['categoria'] = df['valor'].apply(lambda x: 'Alto' if x > 100 else 'Bajo')

# Transformacion tipo de dato
df['col_numerica'] = df['col_str'].astype(float)
df['col_str'] = df['col_num'].astype(str)
df['col_int'] = df['col_float'].astype(int)

# Renombrar columnas
df.rename(columns={'old_name': 'new_name'}, inplace=True)
df.columns = ['col1', 'col2', 'col3']    # renombrar todas

# Crear columna con enumerate (agregar ID)
df['ID'] = list(range(len(df)))

## 10. GroupBy y Agregaciones (CLAVE EXAMENES)

In [ ]:
# GroupBy basico
df_grouped = df.groupby('columna')['valor'].sum()
df_grouped = df.groupby('columna')['valor'].mean()

# GroupBy con as_index=False (devuelve DataFrame, no Series)
df_agg = df.groupby('columna', as_index=False)['valor'].count()

# Multiples columnas de agrupacion (PATRON EXAMENES)
df_agg = df.groupby(['company', 'issue'], as_index=False)['product'].count()

# === FUNCIONES DE AGREGACION ===

# count() - cuenta valores NO nulos
df.groupby('col')['valor'].count()

# size() - cuenta TODAS las filas (incluye NAs)
df.groupby('col').size()

# sum() - suma de valores
df.groupby('col')['ventas'].sum()

# mean() - promedio (ignora NAs)
df.groupby('col')['rating'].mean()

# median() - mediana
df.groupby('col')['price'].median()

# std() - desviacion estandar
df.groupby('col')['score'].std()

# var() - varianza
df.groupby('col')['score'].var()

# min() / max() - minimo y maximo
df.groupby('col')['age'].min()
df.groupby('col')['age'].max()

# first() / last() - primer y ultimo valor
df.groupby('col')['name'].first()
df.groupby('col')['name'].last()

# nunique() - cuenta valores unicos
df.groupby('company')['product'].nunique()

# quantile() - percentiles
df.groupby('col')['price'].quantile(0.75)  # percentil 75

# agg() con multiples funciones
df.groupby('col')['valor'].agg(['mean', 'std', 'min', 'max'])

# Named aggregation (MEJOR OPCION - EXAMEN 2023)
df_result = df.groupby(['company', 'issue'], as_index=False).agg(
    total=('product', 'size'),           # cuenta total de filas
    timely_sum=('timely_response', 'sum'),  # suma de 1s y 0s
    dispute_sum=('consumer_disputed', 'sum'),
    avg_rating=('rating', 'mean'),       # promedio
    max_price=('price', 'max'),          # maximo
    unique_products=('product', 'nunique')  # conteo de unicos
)

# Derivar columnas con assign despues de agg
df_result = df_result.assign(
    untimely_pct=lambda d: (d.total - d.timely_sum) / d.total * 100,
    disputed_pct=lambda d: d.dispute_sum / d.total * 100
)

# Agregacion con lambdas personalizadas
df.groupby('col').agg(
    rango=('valor', lambda x: x.max() - x.min()),
    p90=('valor', lambda x: x.quantile(0.9))
)

## 10b. Agregaciones Avanzadas

In [ ]:
# Diferentes agregaciones por columna
df.groupby('categoria').agg({
    'ventas': ['sum', 'mean'],
    'precio': ['min', 'max'],
    'cantidad': 'count'
})

# Aplanar MultiIndex columns despues de agg multiple
df_agg = df.groupby('col').agg({'val1': ['sum', 'mean'], 'val2': 'count'})
df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]
df_agg.reset_index(inplace=True)

# Filtrar grupos con .filter()
# Solo grupos donde la media > 100
df_filtered = df.groupby('categoria').filter(lambda x: x['valor'].mean() > 100)

# Transformar (mantiene el tamaño original, difunde valores)
# Asignar la media del grupo a cada fila
df['media_grupo'] = df.groupby('categoria')['valor'].transform('mean')

# Diferencia respecto a la media del grupo
df['diff_media'] = df['valor'] - df.groupby('categoria')['valor'].transform('mean')

# Ranking dentro de cada grupo
df['rank'] = df.groupby('categoria')['valor'].rank(ascending=False)

# Acumulado por grupo
df['cumsum'] = df.groupby('categoria')['valor'].cumsum()

# apply() para operaciones complejas (retorna agregado o transformado)
df.groupby('col').apply(lambda x: x.nlargest(3, 'valor'))  # top 3 por grupo

## 10c. Casos Practicos de Agregacion (Examenes)

In [ ]:
# CASO 1: Top 5 empresas con mas quejas (EXAMEN 2023)
top_companies = (df.groupby('company', as_index=False)['issue']
                   .count()
                   .sort_values('issue', ascending=False)
                   .head(5))

# CASO 2: Top 3 issues por volumen (EXAMEN 2023)
top_issues = (df.groupby('issue', as_index=False)['product']
                .count()
                .sort_values('product', ascending=False)
                .head(3))

# CASO 3: Filtrar por top issues
df_filtered = df[df['issue'].isin(top_issues['issue'])]

# CASO 4: Metricas por empresa e issue (EXAMEN 2023)
# Calcular % de respuestas no oportunas y % disputas
result = (df.groupby(['company', 'issue'], as_index=False)
            .agg(
                total=('product', 'size'),
                timely_yes=('timely_response', 'sum'),  # suma de 1s
                disputed=('consumer_disputed', 'sum')
            )
            .assign(
                untimely_pct=lambda d: (1 - d.timely_yes/d.total) * 100,
                disputed_pct=lambda d: d.disputed / d.total * 100
            )
            .drop(columns=['timely_yes', 'disputed']))

# CASO 5: Contar peliculas por decada y genero (EXAMEN 2022)
df['decade'] = (df['Year'] // 10) * 10
counts = (df.groupby(['decade', 'main_genre'], as_index=False)
            .size()
            .rename(columns={'size': 'count'})
            .sort_values(['decade', 'count'], ascending=False))

# CASO 6: Top 3 generos por decada (EXAMEN 2022)
top_by_decade = pd.DataFrame()
for decade in df['decade'].unique():
    top_3 = counts[counts['decade'] == decade].head(3)
    top_by_decade = pd.concat([top_by_decade, top_3], ignore_index=True)

# CASO 7: Generos con mas de 200 apariciones
genre_counts = df.groupby('main_genre')['decade'].count()
valid_genres = genre_counts[genre_counts > 200].index
df_filtered = df[df['main_genre'].isin(valid_genres)]

## 11. Sort y Top N

In [ ]:
# Ordenar por una columna
df_sorted = df.sort_values(by='columna', ascending=False)

# Ordenar por multiples columnas (PATRON EXAMEN 2022)
df_sorted = df.sort_values(by=['decade', 'count'], ascending=False)
df_sorted = df.sort_values(by=['col1', 'col2'], ascending=[True, False])

# Ordenar y reset index
df.sort_values(by='col', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

# Top N valores (PATRON TODOS LOS EXAMENES)
top_5 = df.sort_values(by='count', ascending=False).head(5)
top_5 = df.sort_values(by='count', ascending=False)[:5]  # slice

# Top N por grupo (EXAMEN 2023 - top 3 issues)
top_issues = (df.groupby('issue', as_index=False)['product']
              .count()
              .sort_values('product', ascending=False)
              .head(3))

# nlargest / nsmallest (alternativa rapida)
df.nlargest(5, 'columna')
df.nsmallest(3, 'columna')

## 12. Concatenar y Merge DataFrames

In [ ]:
# Concatenar verticalmente (apilar filas - PATRON EXAMEN 2022)
df_total = pd.DataFrame()
for grupo in df['categoria'].unique():
    df_subset = df[df['categoria'] == grupo].head(3)
    df_total = pd.concat([df_total, df_subset], ignore_index=True)

# Concatenar horizontalmente (columnas)
df_combined = pd.concat([df1, df2], axis=1)

# Append (deprecado, usar concat)
df_total = pd.concat([df1, df2], ignore_index=True)

# Merge (join) - como SQL
df_merged = pd.merge(df1, df2, on='key_column', how='inner')
# how: 'inner', 'left', 'right', 'outer'

# Merge con diferentes nombres de columna
df_merged = pd.merge(df1, df2, left_on='col1', right_on='col2', how='left')

# Merge por indice
df_merged = pd.merge(df1, df2, left_index=True, right_index=True)

## 13. Iteracion sobre Filas/Columnas

In [ ]:
# Iterar sobre grupos (PATRON EXAMEN 2022 - por decada)
for categoria, grupo_df in df.groupby('categoria'):
    print(f'Procesando: {categoria}')
    top_3 = grupo_df.nlargest(3, 'valor')
    # hacer algo con top_3

# Iterar sobre valores unicos
for valor in df['columna'].unique():
    df_subset = df[df['columna'] == valor]
    # procesar df_subset

# Iterar sobre filas (EVITAR si es posible, preferir apply)
for index, row in df.iterrows():
    print(row['columna1'], row['columna2'])

# Iterar sobre columnas
for col in df.columns:
    print(f'{col}: tipo {df[col].dtype}')

# List comprehension (mas eficiente)
lista_valores = [valor * 2 for valor in df['columna']]

## 14. Pivoting y Reshaping

In [ ]:
# Pivot table (formato ancho)
pivot = df.pivot_table(
    values='valor',           # columna a agregar
    index='fila',             # columna para filas
    columns='columna',        # columna para columnas
    aggfunc='mean'            # funcion agregacion
)

# Crosstab (tabla de frecuencias)
ct = pd.crosstab(df['col1'], df['col2'])

# Melt (formato largo - opuesto a pivot)
df_long = df.melt(
    id_vars=['id', 'name'],   # columnas que no cambian
    value_vars=['col1', 'col2'],  # columnas a transformar
    var_name='variable',      # nombre para la nueva columna de variables
    value_name='valor'        # nombre para la columna de valores
)

# Transpose
df_T = df.T                   # intercambiar filas y columnas

## 15. Tips y Trucos Finales

In [ ]:
# Copiar DataFrame (evitar referencias)
df_copy = df.copy()           # copia profunda

# Sampling
df_sample = df.sample(n=100)  # 100 filas aleatorias
df_sample = df.sample(frac=0.1)  # 10% de filas

# Query (filtrado con strings)
df_filtrado = df.query('Year >= 1980 and Year <= 2019')
df_filtrado = df.query('main_genre in ["Action", "Drama"]')

# Conditional assignment (np.where)
df['categoria'] = np.where(df['valor'] > 100, 'Alto', 'Bajo')

# Multiples condiciones con np.select
conditions = [df['valor'] < 50, df['valor'] < 100, df['valor'] >= 100]
choices = ['Bajo', 'Medio', 'Alto']
df['categoria'] = np.select(conditions, choices, default='Otro')

# Display settings (mostrar mas filas/columnas)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Memory usage
df.memory_usage(deep=True).sum() / 1024**2  # MB

## 16. Binning y Discretizacion (qcut, cut)

In [ ]:
# pd.qcut() - divide en grupos de IGUAL TAMAÑO (cuantiles)
# PATRON EXAMEN 01MIAR - dividir por edad en grupos iguales
df['AgeGroup'] = pd.qcut(df['Age'], q=5, labels=['E1','E2','E3','E4','E5'])

# qcut con duplicates='drop' si hay valores repetidos
df['AgeGroup'] = pd.qcut(df['Age'], q=5, labels=['E1','E2','E3','E4','E5'], 
                         duplicates='drop')

# pd.cut() - divide por RANGOS especificos (intervalos fijos)
bins = [0, 18, 30, 50, 100]
labels = ['Menor', 'Joven', 'Adulto', 'Mayor']
df['AgeCategory'] = pd.cut(df['Age'], bins=bins, labels=labels)

# cut con right=False (incluye limite izquierdo, excluye derecho)
df['Score_Cat'] = pd.cut(df['score'], bins=[0, 50, 75, 100], 
                         labels=['Bajo', 'Medio', 'Alto'], right=False)

# Ejemplo completo: qcut para percentiles
df['Quartile'] = pd.qcut(df['price'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Ver rangos creados
df.groupby('AgeGroup')['Age'].agg(['min', 'max', 'count'])

## 17. Correlaciones (EXAMEN 01MIAR)

In [ ]:
# Matriz de correlacion completa
corr_matrix = df.corr()
print(corr_matrix)

# Correlacion entre dos columnas especificas
corr_value = df['col1'].corr(df['col2'])

# Correlaciones con una columna target
correlations = df.corr()['target_col']
print(correlations)

# Verificar correlaciones > threshold (PATRON EXAMEN 01MIAR)
def verify_corr(df, target):
    corr_matrix = df.corr()
    correlations = corr_matrix[target]
    
    result = {}
    for col in df.columns:
        if col != target:
            # True si |correlacion| > 0.3
            result[col] = abs(correlations[col]) > 0.3
    
    return result

# Uso
corr_dict = verify_corr(df, 'Fare')
print(corr_dict)  # {'Pclass': True, 'Age': False, 'Relatives': False}

# Correlaciones solo de columnas numericas
df_numeric = df.select_dtypes(include=[np.number])
corr_matrix = df_numeric.corr()

# Encontrar correlaciones altas (excluyendo diagonal)
# Correlaciones > 0.7 (positivas o negativas)
high_corr = corr_matrix[(corr_matrix > 0.7) | (corr_matrix < -0.7)]

## 18. Generadores con Pandas (EXAMEN 01MIAR)

In [ ]:
# Pipeline de generadores para leer CSV (EXAMEN 01MIAR)
file_name = "archivo.csv"
lines = (line for line in open(file_name))           # generador de lineas
list_line = (s.rstrip().split("|") for s in lines)  # generador de listas
cols = next(list_line)                               # primera linea = columnas
passengers = (dict(zip(cols, data)) for data in list_line)  # generador de dicts

# Consumir generador
for passenger in passengers:
    print(passenger)

# Convertir generador a DataFrame
passengers_list = list(passengers)
df = pd.DataFrame(passengers_list)

# Generador que divide por valores unicos (EXAMEN 01MIAR)
def divide_by_uniques(df, column):
    unique_values = df[column].unique()
    for value in unique_values:
        df_filtered = df[df[column] == value]
        yield df_filtered

# Uso del generador
gen = divide_by_uniques(df, 'Pclass')
for df_class in gen:
    print(f"Procesando clase: {df_class['Pclass'].iloc[0]}")
    print(f"Filas: {len(df_class)}")

# Generador simple de DataFrames
def generate_chunks(df, chunk_size=100):
    for start in range(0, len(df), chunk_size):
        yield df.iloc[start:start + chunk_size]

# Uso
for chunk in generate_chunks(df, 100):
    # procesar cada chunk
    pass

## 19. Guardar DataFrames por Grupos

In [ ]:
# Guardar DataFrame en CSV (PATRON EXAMEN 01MIAR)
df.to_csv('output.csv', sep='|')                    # con separador |
df.to_csv('output.csv', index=False)                # sin indice
df.to_csv('output.csv', sep='|', index=False)

# Dividir y guardar por grupos (EXAMEN 01MIAR - AgeGroups)
groups = ['E1', 'E2', 'E3', 'E4', 'E5']
df['AgeGroup'] = pd.qcut(df['Age'], q=len(groups), labels=groups, duplicates='drop')

for group in groups:
    df_group = df[df['AgeGroup'] == group]
    df_group.to_csv(f"{group}.csv", sep='|')

# Alternativa con groupby
for name, group_df in df.groupby('AgeGroup'):
    group_df.to_csv(f"{name}.csv", sep='|', index=False)

# Guardar multiple formatos
df.to_csv('data.csv')                               # CSV
df.to_excel('data.xlsx', sheet_name='Sheet1')       # Excel
df.to_json('data.json', orient='records')           # JSON
df.to_parquet('data.parquet')                       # Parquet (mas eficiente)
df.to_pickle('data.pkl')                            # Pickle (preserva tipos)

## 20. Operaciones con Index

In [ ]:
# Reset index (IMPORTANTISIMO despues de filtros/dropna)
df.reset_index(drop=True, inplace=True)  # drop=True: elimina index viejo
df.reset_index(inplace=True)             # mantiene index viejo como columna

# Set index (convertir columna en index)
df.set_index('columna', inplace=True)
df.set_index(['col1', 'col2'], inplace=True)  # multi-index

# Acceder por index
df.loc[0]                                # por label (index)
df.iloc[0]                               # por posicion (0-based)
df.loc[0:5]                              # slice por label (inclusivo)
df.iloc[0:5]                             # slice por posicion (exclusivo final)

# Acceder por index especifico (EXAMEN 01MIAR)
df.iloc[id_max]                          # fila en posicion id_max
df.iloc[id_max].Nombre                   # valor especifico

# Argmax / Argmin - devuelve POSICION del max/min
id_max = df['Estatura'].argmax()         # posicion del maximo
id_min = df['Estatura'].argmin()         # posicion del minimo
valor_max = df.iloc[id_max]['Estatura']

# Reindexar
df.index = range(len(df))                # nuevo index 0,1,2,...
df.index = df.index + 1                  # empezar desde 1

## 21. Lambda vs Funciones Clasicas (EXAMEN 01MIAR)

In [ ]:
# Lambda en una linea (EXAMEN 01MIAR)
import random
oneline_function = lambda i: sum([random.randint(1,20) if n%2==0 else random.randint(-20,-1) for n in range(i)])

# Convertir a funcion legible (EXAMEN 01MIAR)
def readable_function(iterations):
    list_a = []
    for n in range(iterations):
        if n % 2 == 0:
            list_a.append(random.randint(1, 20))
        else:
            list_a.append(random.randint(-20, -1))
    return sum(list_a)

# Lambda con pandas apply
df['categoria'] = df['valor'].apply(lambda x: 'Alto' if x > 100 else 'Bajo')

# Equivalente con funcion
def categorizar(valor):
    if valor > 100:
        return 'Alto'
    else:
        return 'Bajo'

df['categoria'] = df['valor'].apply(categorizar)

# List comprehension vs lambda
# List comprehension (mas rapido)
valores = [x * 2 for x in df['columna']]

# Lambda con map
valores = list(map(lambda x: x * 2, df['columna']))

# Cuando usar cada uno:
# - Lambda: operaciones simples en una linea
# - Funcion: logica compleja, reutilizable, mas legible
# - List comprehension: mejor rendimiento para listas

## 22. Patrones Comunes de Examenes - Workflow Completo

In [ ]:
# === PATRON 1: Leer con generador, calcular porcentajes por grupo ===
# (EXAMEN 01MIAR - Ejercicio 03)
def calc_percent_survived(passengers):
    dict_list = list(passengers)                    # convertir generador
    df = pd.DataFrame(dict_list)
    df["Pclass"] = df["Pclass"].astype(int)
    df["Survived"] = df["Survived"].astype(int)
    
    survived_by_class = df.groupby('Pclass')['Survived'].sum()
    total_by_class = df.groupby('Pclass').size()
    percent = (survived_by_class / total_by_class * 100).round().astype(int)
    
    return pd.DataFrame({'PClass': percent.index, 'PercentSurvived': percent.values})

# === PATRON 2: Dividir en grupos iguales y guardar ===
# (EXAMEN 01MIAR - Ejercicio 04)
def split_dataframe(file_name, groups):
    df = pd.read_csv(file_name, sep="|")
    df = df.dropna(subset=['Age'])
    df = df.sort_values(by='Age').reset_index(drop=True)
    df['AgeGroup'] = pd.qcut(df['Age'], q=len(groups), labels=groups, duplicates='drop')
    
    for group in groups:
        df[df['AgeGroup'] == group].to_csv(f"{group}.csv", sep='|')

# === PATRON 3: Crear columnas derivadas y verificar correlaciones ===
# (EXAMEN 01MIAR - Ejercicio 05)
def read_clean_df(file_name):
    df = pd.read_csv(file_name, sep="|")
    df['Relatives'] = df['SibSp'] + df['Parch']
    return df[['Pclass', 'Age', 'Relatives', 'Fare']]

def verify_corr(df, target):
    corr_matrix = df.corr()
    return {col: abs(corr_matrix[target][col]) > 0.3 
            for col in df.columns if col != target}

def divide_by_uniques(df, column):
    for value in df[column].unique():
        yield df[df[column] == value]

# === PATRON 4: Top N empresas, filtrar y metricas ===
# (EXAMEN 2023)
top_5 = df.groupby('company', as_index=False)['issue'].count().sort_values('issue', ascending=False).head(5)
df_top = df[df['company'].isin(top_5['company'])]

result = (df_top.groupby(['company', 'issue'], as_index=False)
          .agg(total=('product', 'size'), timely=('timely_response', 'sum'))
          .assign(untimely_pct=lambda d: (1 - d.timely/d.total) * 100))